In [1]:
import numpy as np
import pandas as pd
import gc
import time
import random

from scipy.stats import wasserstein_distance
from sklearn.preprocessing import StandardScaler


from pandas.api.types import is_numeric_dtype
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit

In [2]:
from sklearn.decomposition import PCA

In [3]:
pd.set_option("display.max_rows", 101)

In [4]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [5]:
df_trans = pd.read_csv(f'{folder_path}train_transaction.csv')#.iloc[:6000,:]
df_id = pd.read_csv(f'{folder_path}train_identity.csv')#.iloc[:6000,:]
df_train = pd.merge(df_trans, df_id, how='left', on='TransactionID')
del df_trans
del df_id

df_trans = pd.read_csv(f'{folder_path}test_transaction.csv')#.iloc[:1000,:]
df_id = pd.read_csv(f'{folder_path}test_identity.csv')#.iloc[:1000,:]
df_test = pd.merge(df_trans, df_id, how='left', on='TransactionID')
del df_trans
del df_id

na_exist_columns = []
for col in df_test.columns:
    if df_train[col].isnull().sum()>0:
        na_exist_columns.append(col)
    else:
        if df_test[col].isnull().sum()>0:
            na_exist_columns.append(col)
            
# train_isnull = df_train[na_exist_columns].isnull().astype(int)
# train_isnull.columns = [f'{col}_isna' for col in train_isnull.columns]
# test_isnull = df_test[na_exist_columns].isnull().astype(int)
# test_isnull.columns = [f'{col}_isna' for col in test_isnull.columns]

for col in df_test.columns:
    try:
        if not is_numeric_dtype(df_test[col]):
            le = LabelEncoder()
            le.fit(df_train[col].fillna('').tolist() + df_test[col].fillna('').tolist())
            df_train[col] = le.transform(df_train[col].fillna(''))
            df_test[col] = le.transform(df_test[col].fillna(''))
        else:
            min_ = np.min([df_train[col].dropna().min(), df_test[col].dropna().min()])
            max_ = np.max([df_train[col].dropna().max(), df_test[col].dropna().max()])
            replace_value = min_- (max_ - min_)*.1
            df_train[col] = df_train[col].fillna(replace_value)
            df_test[col] = df_test[col].fillna(replace_value)
    except:
        raise Exception(f'{col}-{df_train[col].dtype}')
        
for col in df_train.columns:
     if not is_numeric_dtype(df_train[col]):
        print(col, df_train[col].dtype)
        
for col in df_test.columns:
     if not is_numeric_dtype(df_test[col]):
        print(col, df_test[col].dtype)

In [6]:
def emd_select_features(df, features, n_selected=None, label_name='label'):
    """
    this function is for normal detection or binary classification
    :param df:
    :param features:
    :param n_selected:
    :return:
    """

    df_columns_ = df.columns.tolist()
    assert label_name in df_columns_, 'df do not have label column'
    assert label_name not in features, 'label is in features'



    unique_label = df[label_name].unique()
    assert len(unique_label) == 2, 'label must have 2 different values'

    df_norm = pd.DataFrame(StandardScaler().fit_transform(df[features].values), columns=features)
    df_norm[label_name] = df[label_name]
    emd_ranking = []
    for col in features:
        x1_ = df_norm[df_norm[label_name]==unique_label[0]][col].values
        x2_ = df_norm[df_norm[label_name] == unique_label[1]][col].values
        emd = wasserstein_distance(x1_, x2_)
        emd_ranking.append({'feature':col, 'emd':emd, 'type':str(df_norm[col].dtype)})

    if type(n_selected) == type(None):
        n_selected = len(features)
    df_emd_ranking = pd.DataFrame(emd_ranking)
    df_emd_ranking = df_emd_ranking.sort_values(by=['emd'], ascending=False).head(n_selected)

    return df_emd_ranking
    

In [7]:
columns = df_train.columns.drop(['isFraud'])

In [18]:
df_emd_ranking = emd_select_features(df_train, columns, label_name='isFraud')

In [20]:
df_emd_ranking.sort_values(by=['emd'], ascending=False).head(20)

,feature,emd,type
254,V201,1.010422,float64
253,V200,1.004598,float64
224,V171,0.955881,float64
223,V170,0.949283,float64
295,V242,0.926925,float64
242,V189,0.916205,float64
297,V244,0.909653,float64
241,V188,0.887528,float64
11,addr2,0.878590,float64
409,id_17,0.877526,float64


In [8]:
cate_cols = []
for col in df_train.columns:
    if not 'float' in str(df_train[col].dtype):
        print(col, df_train[col].dtype, df_train[col].nunique())
        cate_cols.append(col)

TransactionID int64 590540
isFraud int64 2
TransactionDT int64 573349
ProductCD int64 5
card1 int64 13553
card4 int64 5
card6 int64 5
P_emaildomain int64 60
R_emaildomain int64 61
M1 int64 3
M2 int64 3
M3 int64 3
M4 int64 4
M5 int64 3
M6 int64 3
M7 int64 3
M8 int64 3
M9 int64 3
id_12 int64 3
id_15 int64 4
id_16 int64 3
id_23 int64 4
id_27 int64 3
id_28 int64 3
id_29 int64 3
id_30 int64 76
id_31 int64 131
id_33 int64 261
id_34 int64 5
id_35 int64 3
id_36 int64 3
id_37 int64 3
id_38 int64 3
DeviceType int64 3
DeviceInfo int64 1787


In [9]:
strong_corr_with_v201 = ['V169',
 'V170',
 'V171',
 'V172',
 'V173',
 'V175',
 'V176',
 'V180',
 'V181',
 'V183',
 'V184',
 'V185',
 'V186',
 'V188',
 'V189',
 'V190',
 'V191',
 'V194',
 'V195',
 'V196',
 'V197',
 'V198',
 'V199',
 'V200',
 'V201',
 'V202',
 'V203',
 'V204',
 'V205',
 'V206',
 'V207',
 'V208',
 'V209',
 'V210',
 'V211',
 'V212',
 'V213',
 'V214',
 'V215',
 'V216',
 'V220',
 'V221',
 'V222',
 'V226',
 'V227',
 'V228',
 'V229',
 'V230',
 'V238',
 'V239',
 'V240',
 'V241',
 'V242',
 'V243',
 'V244',
 'V245',
 'V246',
 'V247',
 'V249',
 'V250',
 'V251',
 'V252',
 'V255',
 'V256',
 'V257',
 'V258',
 'V259',
 'V260',
 'V261',
 'V262',
 'V263',
 'V264',
 'V265',
 'V266',
 'V267',
 'V268',
 'V269',
 'V270',
 'V271',
 'V272',
 'V273',
 'V274',
 'V275',
 'V276',
 'V277',
 'V278',
 'V302',
 'id_01',
 'id_05',
 'id_06',
 'id_11',
 'id_12',
 'id_17',
 'id_35',
 'id_36',
 'id_37']

In [21]:
# dff_corr = pd.DataFrame(StandardScaler().fit_transform(pd.concat(df_train.values), columns=df_train.columns.tolist()).corr()

In [48]:
# ccolumns = [col for col in dff_corr.columns if 'V' in col]

In [51]:
# strong_corr_with_v201 = dff_corr[(dff_corr['V201']>.9)|(dff_corr['V201']<-.9)][['V201']].index.tolist()

In [10]:
pca = PCA(n_components=len(strong_corr_with_v201))

In [11]:
ss = StandardScaler()
X_v201_pca = pca.fit_transform(ss.fit_transform(np.vstack((df_train[strong_corr_with_v201].values, df_test[strong_corr_with_v201].values))))

In [12]:
X_v201_pca[:df_train.shape[0],:].shape, X_v201_pca[df_train.shape[0]:,:].shape, df_train.shape, df_test.shape

((590540, 96), (506691, 96), (590540, 434), (506691, 433))

In [13]:
df_v201_coms_train = pd.DataFrame(X_v201_pca[:df_train.shape[0],:], columns=[f'v201_pca_com_{i}' for i in np.arange(X_v201_pca.shape[1])])

In [14]:
df_v201_coms_test = pd.DataFrame(X_v201_pca[:df_train.shape[0],:], columns=[f'v201_pca_com_{i}' for i in np.arange(X_v201_pca.shape[1])])

In [21]:
top30_coms = ['v201_pca_com_31',
 'v201_pca_com_19',
 'v201_pca_com_54',
 'v201_pca_com_67',
 'v201_pca_com_51',
 'v201_pca_com_4',
 'v201_pca_com_49',
 'v201_pca_com_40',
 'v201_pca_com_36',
 'v201_pca_com_65',
 'v201_pca_com_56',
 'v201_pca_com_44',
 'v201_pca_com_71',
 'v201_pca_com_38',
 'v201_pca_com_59',
 'v201_pca_com_45',
 'v201_pca_com_43',
 'v201_pca_com_55',
 'v201_pca_com_76',
 'v201_pca_com_46',
 'v201_pca_com_52',
 'v201_pca_com_78',
 'v201_pca_com_63',
 'v201_pca_com_35',
 'v201_pca_com_32',
 'v201_pca_com_60',
 'v201_pca_com_66',
 'v201_pca_com_34',
 'v201_pca_com_61',
 'v201_pca_com_21']

In [22]:
df_train = pd.concat([df_train, df_v201_coms_train], axis=1)
df_test = pd.concat([df_test, df_v201_coms_test], axis=1)

In [23]:
cate_cols = ['ProductCD',
 'card1',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_12',
 'id_15',
 'id_16',
 'id_23',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

In [25]:
for float_value in top30_coms:
    for col in cate_cols:
        mean_train = df_train.groupby([col])[float_value].transform('mean')
        df_train[f'{float_value}_f{col}_diff'] =  df_train[float_value] / mean_train
        df_train[f'{float_value}_f{col}_rate'] =  df_train[float_value] - mean_train
        
        mean_test = df_test.groupby([col])[float_value].transform('mean')
        df_test[f'{float_value}_f{col}_diff'] =  df_test[float_value] / mean_test
        df_test[f'{float_value}_f{col}_rate'] =  df_test[float_value] - mean_test

In [26]:
new_cols = []
for col in df_train.columns:
    if (col.endswith('diff')) |(col.endswith('rate')):
        new_cols.append(col)

In [27]:
df_train = df_train.loc[:,~df_train.columns.duplicated()]
df_test = df_test.loc[:,~df_test.columns.duplicated()]

In [28]:
df_emd_ranking = emd_select_features(df_train, new_cols, label_name='isFraud')

In [30]:
df_emd_ranking.head(20).feature

,feature,emd,type
960,v201_pca_com_45_fProductCD_diff,1.359564,float64
128,v201_pca_com_54_fProductCD_diff,1.354705,float64
448,v201_pca_com_40_fProductCD_diff,1.343214,float64
1088,v201_pca_com_55_fProductCD_diff,1.284924,float64
990,v201_pca_com_45_fid_12_diff,1.266673,float64
486,v201_pca_com_40_fid_27_diff,1.266589,float64
484,v201_pca_com_40_fid_23_diff,1.265484,float64
451,v201_pca_com_40_fcard1_rate,1.261802,float64
135,v201_pca_com_54_fcard6_rate,1.259797,float64
183,v201_pca_com_54_fid_36_rate,1.259280,float64


In [32]:
df_emd_ranking.head(20).feature.tolist()

['v201_pca_com_45_fProductCD_diff',
 'v201_pca_com_54_fProductCD_diff',
 'v201_pca_com_40_fProductCD_diff',
 'v201_pca_com_55_fProductCD_diff',
 'v201_pca_com_45_fid_12_diff',
 'v201_pca_com_40_fid_27_diff',
 'v201_pca_com_40_fid_23_diff',
 'v201_pca_com_40_fcard1_rate',
 'v201_pca_com_54_fcard6_rate',
 'v201_pca_com_54_fid_36_rate',
 'v201_pca_com_54_fid_33_rate',
 'v201_pca_com_54_fid_37_rate',
 'v201_pca_com_54_fR_emaildomain_rate',
 'v201_pca_com_54_fid_15_rate',
 'v201_pca_com_54_fid_27_rate',
 'v201_pca_com_54_fid_12_rate',
 'v201_pca_com_54_fid_28_rate',
 'v201_pca_com_54_fid_16_rate',
 'v201_pca_com_54_fid_29_rate',
 'v201_pca_com_54_fid_38_rate']

In [35]:
df_train[['TransactionID']+df_emd_ranking.head(20).feature.tolist()].to_pickle(f'{folder_path}train_v201_components', compression='gzip')

In [36]:
df_test[['TransactionID']+df_emd_ranking.head(20).feature.tolist()].to_pickle(f'{folder_path}test_v201_components', compression='gzip')

In [ ]:
category_ranking = {}
category_cols = []
columns = []
feature = ''
depth = 2
label = ''
generation_size = 10
generation = 10

for _ in range(generation):
    for _ in range(generation_size):

        selected_category_cols = random.sample(category_cols, depth)
        selected_category_cols_string = '_'.join(selected_category_cols)
        mean_train = df_train.groupby(selected_category_cols)[feature].transform('mean')
        std_train = df_train.groupby(selected_category_cols)[feature].transform('mean')
        df_train[f'{feature}_{selected_category_cols_string}_rate'] =  df_train[float_value] / mean_train
        df_train[f'{feature}_{selected_category_cols_string}_diff'] =  df_train[float_value] - mean_train
        df_train[f'{feature}_{selected_category_cols_string}_mean'] =  mean_train
        df_train[f'{feature}_{selected_category_cols_string}_std'] =  std_train

        mean_test = df_test.groupby(selected_category_cols)[feature].transform('mean')
        std_test = df_test.groupby(selected_category_cols)[feature].transform('mean')
        df_test[f'{feature}_{selected_category_cols_string}_rate'] =  df_test[float_value] / mean_test
        df_test[f'{feature}_{selected_category_cols_string}_diff'] =  df_test[float_value] - mean_test
        df_test[f'{feature}_{selected_category_cols_string}_mean'] =  mean_test
        df_test[f'{feature}_{selected_category_cols_string}_std'] =  std_test

        new_feats = [f'{feature}_{selected_category_cols_string}_rate',
        f'{feature}_{selected_category_cols_string}_diff',
        f'{feature}_{selected_category_cols_string}_mean',
        f'{feature}_{selected_category_cols_string}_std']

        df_train[new_feats+[label]]
        df_emd_ranking = emd_select_features(df_train[new_feats+[label]], new_feats, label_name=label)
        df_emd_ranking.sort_values(by=['emd'], ascending=False).head(20)
        max_emd = df_emd_ranking['emd'].max()
        for col in selected_category_cols:
            category_ranking[col] = category_ranking[col]+max_emd
